In [31]:
import torch
import gradientUtils as gu

# Define a threshold for early stopping
threshold = 1e-16
# Define the ground truth vertex
X_g = torch.tensor([1.0], requires_grad=False)
Y_g = torch.tensor([1.0], requires_grad=False)
# Initialize the sites
s_i = gu.Site(1.0, 1.0)
s_j = gu.Site(2.0, 2.0)
s_k = gu.Site(1.50, 2.750)
destination = "images/autograd/"

### AUTO GRAD

In [32]:
def autograd(s_i, s_j, s_k, X_g, Y_g, destination):
    # Define an optimizer
    optimizer = torch.optim.Adam([s_i.x, s_i.y, s_j.x, s_j.y, s_k.x, s_k.y], lr=0.01)
    # Previous loss
    prev_loss = float('inf')
    # Training loop
    for epoch in range(1000):
        # Zero the gradients
        optimizer.zero_grad()

        # Compute the vertex
        X, Y = gu.compute_vertex(s_i, s_j, s_k)

        # Compute the loss
        loss = (X - X_g) ** 2 + (Y - Y_g) ** 2
        # Early stopping condition
        if abs(prev_loss - loss.item()) < threshold:
            print(f"Stopping early at epoch {epoch} due to minimal loss change")
            gu.plot_and_save(epoch, s_i, s_j, s_k, X_g, Y_g, destination)
            break

        prev_loss = loss.item()
        # Backpropagate the error
        loss.backward()

        # Update the parameters
        optimizer.step()

        # Print the loss
        if epoch % 100 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')
            gu.plot_and_save(epoch, s_i, s_j, s_k, X_g, Y_g, destination)

    # Print the final site coordinates
    print(f'Final site coordinates:')
    print(f's_i: ({s_i.x.item()}, {s_i.y.item()})')
    print(f's_j: ({s_j.x.item()}, {s_j.y.item()})')
    print(f's_k: ({s_k.x.item()}, {s_k.y.item()})')

    # Compute the final vertex
    final_X, final_Y = gu.compute_vertex(s_i, s_j, s_k)

    # Print the final vertex coordinates
    print(f'Final vertex coordinates: ({final_X.item()}, {final_Y.item()})')
    return s_i, s_j, s_k


s_i, s_j, s_k = autograd(s_i, s_j, s_k, X_g, Y_g, destination)

Epoch 0, Loss: 0.8612499237060547
Epoch 100, Loss: 0.0021477900445461273
Epoch 200, Loss: 5.845098627332845e-09
Stopping early at epoch 279 due to minimal loss change
Final site coordinates:
s_i: (0.10342850536108017, 0.35356053709983826)
s_j: (1.8965137004852295, 1.6465104818344116)
s_k: (1.1535061597824097, 2.094599962234497)
Final vertex coordinates: (0.9999982714653015, 0.9999975562095642)


### Custom gradient

In [33]:
# Define a threshold for early stopping
threshold = 1e-16
# Define the ground truth vertex
X_g = torch.tensor([1.0], requires_grad=False)
Y_g = torch.tensor([1.0], requires_grad=False)
# Initialize the sites
s_i = gu.Site(1.0, 1.0)
s_j = gu.Site(2.0, 2.0)
s_k = gu.Site(1.50, 2.750)
destination = "images/customgrad"

In [34]:
def customgrad(s_i, s_j, s_k, X_g, Y_g, destination):
    # Define an optimizer
    optimizer = torch.optim.Adam([s_i.x, s_i.y, s_j.x, s_j.y, s_k.x, s_k.y], lr=0.01)
    prev_loss = float('inf')

    class CustomVertexFunction(torch.autograd.Function):
        @staticmethod
        def forward(ctx, x_i, y_i, x_j, y_j, x_k, y_k):
            N_X = x_i ** 2 * (y_j - y_k) - x_j ** 2 * (y_i - y_k) + (x_k ** 2 + (y_i - y_k) * (y_j - y_k)) * (y_i - y_j)
            N_Y = -(x_i ** 2 * (x_j - x_k) - x_i * (
                        x_j ** 2 - x_k ** 2 + y_j ** 2 - y_k ** 2) + x_j ** 2 * x_k - x_j * (
                                x_k ** 2 - y_i ** 2 + y_k ** 2) - x_k * (y_i ** 2 - y_j ** 2))
            D = 2 * (x_i * (y_j - y_k) - x_j * (y_i - y_k) + x_k * (y_i - y_j))

            X = N_X / D
            Y = N_Y / D

            ctx.save_for_backward(x_i, y_i, x_j, y_j, x_k, y_k, X, Y, N_X, N_Y, D)
            return X, Y

        @staticmethod
        def backward(ctx, grad_output_X, grad_output_Y):
            x_i, y_i, x_j, y_j, x_k, y_k, X, Y, N_X, N_Y, D = ctx.saved_tensors

            # Compute partial derivatives of X and Y wrt each site coordinate
            dN_X_dx_i = 2 * x_i * (y_j - y_k)
            dN_X_dx_j = -2 * x_j * (y_i - y_k)
            dN_X_dx_k = 2 * x_k * (y_i - y_j)

            dN_Y_dx_i = -2 * x_i * (x_j - x_k) - x_j ** 2 + x_k ** 2 - y_j ** 2 + y_k ** 2
            dN_Y_dx_j = -x_i ** 2 + x_k ** 2
            dN_Y_dx_k = -x_i ** 2 + x_j ** 2

            dD_dx_i = 2 * (y_j - y_k)
            dD_dx_j = 2 * (y_i - y_k)
            dD_dx_k = 2 * (y_i - y_j)

            dX_dx_i = (dN_X_dx_i * D - N_X * dD_dx_i) / (D ** 2)
            dX_dx_j = (dN_X_dx_j * D - N_X * dD_dx_j) / (D ** 2)
            dX_dx_k = (dN_X_dx_k * D - N_X * dD_dx_k) / (D ** 2)

            dY_dx_i = (dN_Y_dx_i * D - N_Y * dD_dx_i) / (D ** 2)
            dY_dx_j = (dN_Y_dx_j * D - N_Y * dD_dx_j) / (D ** 2)
            dY_dx_k = (dN_Y_dx_k * D - N_Y * dD_dx_k) / (D ** 2)

            # Similarly for the y coordinates
            dN_X_dy_i = 2 * (x_j - x_k) * x_i - x_j ** 2 + x_k ** 2 + y_j ** 2 - y_k ** 2
            dN_X_dy_j = -2 * (x_i - x_k) * x_j + x_i ** 2 - x_k ** 2 - y_i ** 2 + y_k ** 2
            dN_X_dy_k = 2 * (x_i - x_j) * x_k - x_i ** 2 + x_j ** 2 + y_i ** 2 - y_j ** 2

            dN_Y_dy_i = 0
            dN_Y_dy_j = 0
            dN_Y_dy_k = 0

            dD_dy_i = 2 * (x_j - x_k)
            dD_dy_j = 2 * (x_i - x_k)
            dD_dy_k = 2 * (x_i - x_j)

            dX_dy_i = (dN_X_dy_i * D - N_X * dD_dy_i) / (D ** 2)
            dX_dy_j = (dN_X_dy_j * D - N_X * dD_dy_j) / (D ** 2)
            dX_dy_k = (dN_X_dy_k * D - N_X * dD_dy_k) / (D ** 2)

            dY_dy_i = (dN_Y_dy_i * D - N_Y * dD_dy_i) / (D ** 2)
            dY_dy_j = (dN_Y_dy_j * D - N_Y * dD_dy_j) / (D ** 2)
            dY_dy_k = (dN_Y_dy_k * D - N_Y * dD_dy_k) / (D ** 2)

            grad_x_i = grad_output_X * dX_dx_i + grad_output_Y * dY_dx_i
            grad_y_i = grad_output_X * dX_dy_i + grad_output_Y * dY_dy_i
            grad_x_j = grad_output_X * dX_dx_j + grad_output_Y * dY_dx_j
            grad_y_j = grad_output_X * dX_dy_j + grad_output_Y * dY_dy_j
            grad_x_k = grad_output_X * dX_dx_k + grad_output_Y * dY_dx_k
            grad_y_k = grad_output_X * dX_dy_k + grad_output_Y * dY_dy_k

            return grad_x_i, grad_y_i, grad_x_j, grad_y_j, grad_x_k, grad_y_k

    def compute_vertex(s_i, s_j, s_k):
        return CustomVertexFunction.apply(s_i.x, s_i.y, s_j.x, s_j.y, s_k.x, s_k.y)

    # Training loop
    for epoch in range(10000):
        # Zero the gradients
        optimizer.zero_grad()

        # Compute the vertex
        X, Y = compute_vertex(s_i, s_j, s_k)

        # Compute the loss
        loss = (X - X_g) ** 2 + (Y - Y_g) ** 2
        # Early stopping condition
        # if abs(prev_loss - loss.item()) < threshold:
        #     print(f"Stopping early at epoch {epoch} due to minimal loss change")
        #     gu.plot_and_save(epoch, s_i, s_j, s_k, X_g, Y_g, destination)
        #     break

        prev_loss = loss.item()
        # Backpropagate the error
        loss.backward()

        # Update the parameters
        optimizer.step()

        # Print the loss
        if epoch % 100 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')
            gu.plot_and_save(epoch, s_i, s_j, s_k, X_g, Y_g, destination)

    # Print the final site coordinates
    print(f'Final site coordinates:')
    print(f's_i: ({s_i.x.item()}, {s_i.y.item()})')
    print(f's_j: ({s_j.x.item()}, {s_j.y.item()})')
    print(f's_k: ({s_k.x.item()}, {s_k.y.item()})')

    # Compute the final vertex
    final_X, final_Y = compute_vertex(s_i, s_j, s_k)

    # Print the final vertex coordinates
    print(f'Final vertex coordinates: ({final_X.item()}, {final_Y.item()})')
    return s_i, s_j, s_k


s_i, s_j, s_k = customgrad(s_i, s_j, s_k, X_g, Y_g, destination)


Epoch 0, Loss: 0.8612499237060547
Epoch 100, Loss: 0.16594450175762177
Epoch 200, Loss: 0.0030409719329327345
Epoch 300, Loss: 4.169791282038204e-06
Epoch 400, Loss: 2.886579864025407e-11
Epoch 500, Loss: 7.105427357601002e-12
Epoch 600, Loss: 5.9117155615240335e-12
Epoch 700, Loss: 5.8122395785176195e-12
Epoch 800, Loss: 1.7195134205394424e-12
Epoch 900, Loss: 9.237055564881302e-13
Epoch 1000, Loss: 1.1510792319313623e-12
Epoch 1100, Loss: 3.325340003357269e-12
Epoch 1200, Loss: 5.826450433232822e-13
Epoch 1300, Loss: 2.9132252166164108e-12
Epoch 1400, Loss: 4.831690603168681e-13
Epoch 1500, Loss: 9.414691248821327e-13
Epoch 1600, Loss: 1.0373923942097463e-12
Epoch 1700, Loss: 3.872457909892546e-13
Epoch 1800, Loss: 1.2789769243681803e-12
Epoch 1900, Loss: 2.842170943040401e-14
Epoch 2000, Loss: 4.121147867408581e-13
Epoch 2100, Loss: 4.831690603168681e-13
Epoch 2200, Loss: 2.842170943040401e-14
Epoch 2300, Loss: 1.5987211554602254e-13
Epoch 2400, Loss: 3.872457909892546e-13
Epoch 250

In [35]:
# Define a threshold for early stopping
threshold = 1e-16
# Define the ground truth vertex
X_g = torch.tensor([1.0], requires_grad=False)
Y_g = torch.tensor([1.0], requires_grad=False)
# Initialize the sites
s_i = gu.Site(1.0, 1.0)
s_j = gu.Site(-2.0, 1.0)
s_k = gu.Site(3, -3)
destination = "images/autograd/2/"
autograd(s_i, s_j, s_k, X_g, Y_g, destination)

Epoch 0, Loss: 12.8125
Epoch 100, Loss: 1.977822184562683
Epoch 200, Loss: 0.4384814500808716
Epoch 300, Loss: 0.057316653430461884
Epoch 400, Loss: 0.006678937003016472
Epoch 500, Loss: 0.0006953967968001962
Epoch 600, Loss: 5.227408837527037e-05
Epoch 700, Loss: 2.7584478630160447e-06
Epoch 800, Loss: 1.0029931729604868e-07
Epoch 900, Loss: 2.4478516991166543e-09
Stopping early at epoch 985 due to minimal loss change
Final site coordinates:
s_i: (2.416391134262085, 2.1902499198913574)
s_j: (-0.7706083059310913, 0.46346449851989746)
s_k: (1.7692984342575073, -0.6825821399688721)
Final vertex coordinates: (0.9999929070472717, 0.9999938011169434)


(<gradientUtils.Site at 0x7f82ac290700>,
 <gradientUtils.Site at 0x7f82a191f550>)

In [36]:
# Define a threshold for early stopping
threshold = 1e-16
# Define the ground truth vertex
X_g = torch.tensor([1.0], requires_grad=False)
Y_g = torch.tensor([1.0], requires_grad=False)
# Initialize the sites
s_i = gu.Site(1.0, 1.0)
s_j = gu.Site(-2.0, 1.0)
s_k = gu.Site(3, -3)
destination = "images/customgrad/2/"
customgrad(s_i, s_j, s_k, X_g, Y_g, destination)

Epoch 0, Loss: 12.8125
Epoch 100, Loss: 8.707025527954102
Epoch 200, Loss: 0.672167956829071
Epoch 300, Loss: 0.3487200438976288
Epoch 400, Loss: 0.45868346095085144
Epoch 500, Loss: 1.050619125366211
Epoch 600, Loss: 2.3740522861480713
Epoch 700, Loss: 3.048783540725708
Epoch 800, Loss: 2.7810897827148438
Epoch 900, Loss: 2.4095890522003174
Epoch 1000, Loss: 2.2590322494506836
Epoch 1100, Loss: 2.453824996948242
Epoch 1200, Loss: 3.080860137939453
Epoch 1300, Loss: 4.132190227508545
Epoch 1400, Loss: 5.3906145095825195
Epoch 1500, Loss: 6.529107570648193
Epoch 1600, Loss: 7.323753833770752
Epoch 1700, Loss: 7.705397605895996
Epoch 1800, Loss: 7.711126804351807
Epoch 1900, Loss: 7.430761337280273
Epoch 2000, Loss: 6.966740608215332
Epoch 2100, Loss: 6.4090895652771
Epoch 2200, Loss: 5.825100898742676
Epoch 2300, Loss: 5.258607387542725
Epoch 2400, Loss: 4.734340190887451
Epoch 2500, Loss: 4.263580799102783
Epoch 2600, Loss: 3.8492672443389893
Epoch 2700, Loss: 3.489206075668335
Epoch 2

(<gradientUtils.Site at 0x7f82a26dcf40>,
 <gradientUtils.Site at 0x7f82a1f3e4f0>)